<a href="https://colab.research.google.com/github/vvalcristina/alura_apirest/blob/master/Avanco_COVID_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Análise de Dados Covid-19**


Análise das séries temporais sobre a contaminação do COVID-19.Utilizando o DataSet do [Kagle](https://www.kaggle.com/imdevskp/corona-virus-report).

In [0]:
#Importações de dados
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

In [2]:
#Importando o DataSet
df = pd.read_csv("covid_19_clean_complete.csv", parse_dates=['Date'])
df

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.000000,65.000000,2020-01-22,0,0,0
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0
...,...,...,...,...,...,...,...,...
31530,NaN,Sao Tome and Principe,0.186360,6.613081,2020-05-19,251,8,4
31531,NaN,Yemen,15.552727,48.516388,2020-05-19,167,28,5
31532,NaN,Comoros,-11.645500,43.333300,2020-05-19,11,1,3
31533,NaN,Tajikistan,38.861034,71.276093,2020-05-19,1936,41,641


Antes de iniciarmos a análise existem 2 fatores que devemos nos atentar:

* As colunas não podem ter caracteres especiais.

* E nem letras maiusculas. 

In [0]:
#Retirando os caracteres especiais e letras maiusculas do nome das colunas
import re
def corrige_colunas(col_name):
    return re.sub(r"[/| ]", "", col_name).lower()

In [0]:
df.columns = [corrige_colunas(col) for col in df.columns]

In [5]:
#Verificando os dados de coluna
cons = pd.DataFrame({'colunas' : df.columns,
                    'tipo': df.dtypes,
                    'nulos' : df.isna().sum(),
                    'quantidade' : df.shape[0],
                    'unicos': df.nunique()})
cons['percentual'] = round(cons['nulos'] / cons['quantidade'],2)

cons

,colunas,tipo,nulos,quantidade,unicos,percentual
provincestate,provincestate,object,22015,31535,80,0.7
countryregion,countryregion,object,0,31535,188,0.0
lat,lat,float64,0,31535,256,0.0
long,long,float64,0,31535,259,0.0
date,date,datetime64[ns],0,31535,119,0.0
confirmed,confirmed,int64,0,31535,4869,0.0
deaths,deaths,int64,0,31535,1575,0.0
recovered,recovered,int64,0,31535,2907,0.0


Observando as colunas percebemos que apenas a coluna Province/State possui 70% dos seus valores nulos.


**Análises dos dados**


Vamos analisar os dados do Brasil:

* Primeiro passo: investigar as variáveis que temos a nossa disposição. Temos as informaçẽos disponíveis por Estado? 

In [7]:
df.loc[df.countryregion == 'Brazil']

,provincestate,countryregion,lat,long,date,confirmed,deaths,recovered
28,NaN,Brazil,-14.235,-51.9253,2020-01-22,0,0,0
293,NaN,Brazil,-14.235,-51.9253,2020-01-23,0,0,0
558,NaN,Brazil,-14.235,-51.9253,2020-01-24,0,0,0
823,NaN,Brazil,-14.235,-51.9253,2020-01-25,0,0,0
1088,NaN,Brazil,-14.235,-51.9253,2020-01-26,0,0,0
...,...,...,...,...,...,...,...,...
30238,NaN,Brazil,-14.235,-51.9253,2020-05-15,220291,14962,84970
30503,NaN,Brazil,-14.235,-51.9253,2020-05-16,233511,15662,89672
30768,NaN,Brazil,-14.235,-51.9253,2020-05-17,241080,16118,94122
31033,NaN,Brazil,-14.235,-51.9253,2020-05-18,255368,16853,100459


No Brasil não temos a informação de casos de COVID-19 a nível de estado. Verificaremos algumas outras varíaveis para verificar o avanço do COVID-19 no país.

**Casos confirmados**

Verificando os números de casos confirmados

In [0]:
brasil_confirmados = df.loc[(df.countryregion == 'Brazil') & (df.confirmed > 0)]

In [10]:
#Gráficos
px.line(brasil_confirmados, 'date', 'confirmed',
        labels={'date':'Data', 'confirmed':'Numero de casos confirmados'},
        title = 'Casos confirmados de COVID-19 no Brasil')

**Números de casos por dia**

Para verificar os números de novos casos por dia, teremos que implementar uma função

In [11]:
brasil_confirmados['novoscasos'] = list(map(
    lambda x: 0 if (x==0) else brasil_confirmados['confirmed'].iloc[x] - brasil_confirmados['confirmed'].iloc[x-1],
    np.arange(brasil_confirmados.shape[0])
))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [12]:
px.line(brasil_confirmados, x = 'date', y='novoscasos', title='Novos casos de COVID-19 por dia no Brasil',
        labels={'date':'Data', 'novoscasos':'Novos casos'})

**Mortes**

Análise do número de mortes no país pelo COVID-19.

In [16]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=brasil_confirmados.date, y=brasil_confirmados.deaths, name='Mortes',
         mode='lines+markers',line=dict(color='red')))
fig.update_layout(title='Mortes por COVID-19 no Brasil',
                   xaxis_title='Data',
                   yaxis_title='Número de mortes')
fig.show()

Através desde gráfico observa-se que o número de mortos pelo COVID-19 no país segue uma curva exponencial.

In [0]:
def taxa_crescimento_diaria(data, variable, data_inicio=None):
    if data_inicio == None:
        data_inicio = data.date.loc[data[variable] > 0].min()
    else:
        data_inicio = pd.to_datetime(data_inicio)
        
    data_fim = data.date.max()
    n = (data_fim - data_inicio).days
    taxas = list(map(
        lambda x: (data[variable].iloc[x] - data[variable].iloc[x-1]) / data[variable].iloc[x-1],
        range(1,n+1)
    ))
    return np.array(taxas)*100

In [0]:
tx_dia = taxa_crescimento_diaria(brasil_confirmados, 'confirmed')

In [29]:
primeiro_dia = brasil_confirmados.date.loc[brasil_confirmados.confirmed > 0].min()
px.line(x=pd.date_range(primeiro_dia, brasil_confirmados.date.max())[1:],
        y=tx_dia, title='Taxa de crescimento de casos de COVID-19 confirmados no Brasil',
       labels={'y':'Taxa de crescimento', 'x':'Data'})
